 ## <font color=black> PyCity Schools Analysis </font>

Trends Observed:

<br />1 The top 5 performing schools are Charter schools and bottom 5 performing schools are District schools.
<br />2 District schools are larger in size compared to Charter schools.
<br />3 District schools receive higher budget than Charter schools.
<br />4 Despite less funding, Charter school students have higher passin scores than District school students.
<br />5 There does not seem to be significant difference in passing scores by subject when compared by grade levels.


In [122]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [123]:
# Use pd to read files
schools_df = pd.read_csv('./schools_complete.csv')
schools_df.head() 
 



,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [124]:
# Use pd to read files
students_df = pd.read_csv('./students_complete.csv')
students_df.head()


,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


 ## <font color=black> District Summary </font>

In [125]:
# Get the total schools
total_schools = len(schools_df['School ID'])

# Get the total students
total_students = schools_df['size'].sum()

# Get the total budget
total_budget = schools_df['budget'].sum()

# Get the Average Math Score 
avg_math_score = round(students_df["math_score"].mean(), 2)

# Get the Average Reading Score 
avg_reading_score = round(students_df["reading_score"].mean(), 2)

# Calculate the percentage of students passing math
students_passing_math = students_df.loc[students_df["math_score"] >= 70,:]
percent_pass_math = round(float(students_passing_math["math_score"].count()/total_students)*100, 1)

# Calculate the  percentage of students passing reading test
students_passing_read = students_df.loc[students_df["reading_score"] >= 70,:]
percent_pass_read = round(float(students_passing_read["reading_score"].count()/total_students)*100, 1)

# Calculate the overall passing rate for math and reading
overall_pass_rate = round((percent_pass_math + percent_pass_read)/2, 2)

# Create a District Summary DataFrame using the above values
summary_df = pd.DataFrame({"Total Schools": [total_schools],
                          "Total Students": [total_students],
                          "Total Budget": [total_budget],
                          "Average Math Score": [avg_math_score],
                          "Average Reading Score":[avg_reading_score],
                          "% Passing Math": [percent_pass_math],
                          "% Passing Reading": [percent_pass_read],
                          "% Overall Passing Rate": [overall_pass_rate]})
district_summary_df = pd.DataFrame(summary_df, columns=["Total Schools", "Total Students", "Total Budget","Average Math Score",
                                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"])

district_summary_df["Total Students"] = district_summary_df["Total Students"].map('{:,}'.format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map('${:,.2f}'.format)


district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,75.0,85.8,80.4


 ## <font color=black> School Summary </font>

In [113]:
# Change the name of the header in the schools dataframe from name to school
schools_df = schools_df.rename(columns={"name": "school"})
schools_df.columns

# Merge the two DataFrames together based on the school name
schools_data_df = pd.merge(schools_df, students_df, on="school")

# Count the number of students in each school
student_counts = schools_data_df["school"].value_counts()


# Get the school type  
school_type = schools_data_df.groupby('school')['type'].unique()
school_type = school_type.str[0]

# Calculate the total school budget for each school 
each_school_budget = schools_data_df.groupby('school')['budget'].unique()
each_school_budget = each_school_budget.astype(float)

# Calculate each student budgget 
budget_per_student = round(each_school_budget/student_counts,2)
budget_per_student = budget_per_student.astype(float)

# Get the average math and reading scores for each school using groupby and mean functions
school_avg_math_score = round(schools_data_df.groupby('school')['math_score'].mean(),2)
school_avg_reading_score = round(schools_data_df.groupby('school')['reading_score'].mean(),2)

# Create new Dataframe with passing scores for reading and math 
passing_df = schools_data_df.loc[(schools_data_df['math_score'] >= 70) & (schools_data_df['reading_score'] >=70)]
passing_math_df = schools_data_df.loc[(schools_data_df['math_score'] >= 70)]
passing_reading_df = schools_data_df.loc[(schools_data_df['reading_score'] >= 70)]

# Use groupby to get score percentage of students passing math and reading and overall passing rate %
percent_passing_math = round((passing_math_df.groupby('school')['math_score'].count()/student_counts)*100, 1)
percent_passing_reading = round((passing_reading_df.groupby('school')['reading_score'].count()/student_counts)*100, 1)
percent_overall_passing = round((percent_passing_math + percent_passing_reading)/2, 2)

# Creating the School Summary Dataframe based on the given data
school_summary_table = pd.DataFrame({"School Type":school_type,
                                "Total Students":student_counts,
                                "Total School Budget":each_school_budget,
                                "Per Student Budget":budget_per_student,
                                "Average Math Score": school_avg_math_score,
                                "Average Reading Score": school_avg_reading_score,
                                "% Passing Math": percent_passing_math,
                                "% Passing Reading": percent_passing_reading,
                                "% Overall Passing Rate":percent_overall_passing})

# Arrange the columns in order 
school_summary_table = pd.DataFrame(school_summary_table, columns=["School Type", "Total Students", "Total School Budget","Per Student Budget", "Average Math Score",
                                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"])

# Format the df 
school_summary_table["Total Students"] = school_summary_table["Total Students"].map('{:,}'.format)
school_summary_table["Total School Budget"] = school_summary_table["Total School Budget"].map('${:,.2f}'.format)
school_summary_table["Per Student Budget"] = school_summary_table["Per Student Budget"].map('${:,.2f}'.format)

school_summary_table

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.7,81.9,74.30
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.1,97.0,95.55
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,66.0,80.7,73.35
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.3,79.3,73.80
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.4,97.1,95.25
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.8,80.9,73.85
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.5,96.3,94.40
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.7,81.3,73.50
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.1,81.2,73.65
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.6,95.9,95.25


## <font color=black> Top Five Performing Schools (By Percent Passing Rate) </font>

In [114]:
# Arrange schools by passing rate; from high to low
top_5_schools = school_summary_table.sort_values("% Overall Passing Rate", ascending=False, inplace=False)
top_5_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.1,97.0,95.55
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.3,97.3,95.30
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.4,97.1,95.25
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.6,95.9,95.25
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.9,96.5,95.20


## <font color=black> Bottom Five Performing Schools (By Percent Passing Rate) </font>

In [115]:
# Arrange schools by passing rate; from low to high
bottom_5_schools = school_summary_table.sort_values("% Overall Passing Rate", ascending=True, inplace=False)
bottom_5_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.4,80.2,73.30
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,66.0,80.7,73.35
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.7,81.3,73.50
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.1,81.2,73.65
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.3,79.3,73.80


## <font color=black> Math Scores By Grade </font>

In [116]:
# Create grade level average math scores for each school 
ninth_math = students_df.loc[students_df['grade'] == '9th'].groupby('school')["math_score"].mean()
tenth_math = students_df.loc[students_df['grade'] == '10th'].groupby('school')["math_score"].mean()
eleventh_math = students_df.loc[students_df['grade'] == '11th'].groupby('school')["math_score"].mean()
twelfth_math = students_df.loc[students_df['grade'] == '12th'].groupby('school')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

# Display results and format
math_scores.style.format({'9th': '{:.2f}', 
                          "10th": '{:.2f}', 
                          "11th": "{:.2f}", 
                          "12th": "{:.2f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## <font color=black> Reading Scores By Grade </font>

In [117]:

# Create grade level average reading scores for each school
ninth_reading = students_df.loc[students_df['grade'] == '9th'].groupby('school')["reading_score"].mean()
tenth_reading = students_df.loc[students_df['grade'] == '10th'].groupby('school')["reading_score"].mean()
eleventh_reading = students_df.loc[students_df['grade'] == '11th'].groupby('school')["reading_score"].mean()
twelfth_reading = students_df.loc[students_df['grade'] == '12th'].groupby('school')["reading_score"].mean()

# Merge the reading score averages by school and grade
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

# Format the results
reading_scores.style.format({'9th': '{:.2f}', 
                             "10th": '{:.2f}', 
                             "11th": "{:.2f}", 
                             "12th": "{:.2f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## <font color=black> Scores By School Spending </font>

In [118]:
# Create the bins to hold Data
bins = [0, 600, 625, 650, 1000]

# Create the names for the four bins created above
spending_ranges = ["$0-585", "$586-615", "$616-645", "$645-675"]

# Schools_spending_df = school_summary_table
school_summary_table["Spending Ranges (Per Student)"] = pd.cut(budget_per_student, bins=bins, labels=spending_ranges)

# Arrange Dataframe Columns 
spending_math_scores = school_summary_table.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_summary_table.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_summary_table.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_summary_table.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2


scores_by_spend = school_summary_table[["Spending Ranges (Per Student)", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
scores_by_spend = scores_by_spend.groupby("Spending Ranges (Per Student)").mean()

scores_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
$0-585,83.434000,83.894000,93.540000,96.460000,95.000000
$586-615,83.595000,83.930000,94.000000,96.500000,95.250000
$616-645,78.031667,81.416667,71.133333,83.433333,77.283333
$645-675,76.960000,81.055000,66.250000,81.100000,73.675000


## <font color=black> Scores By School Size </font>

In [119]:
# Create the bins to hold Data
bins = [0, 1000, 2000, 5000]

# Create the names for the four bins created above
size_ranges = ["Small (0-1000)", "Medium (1000-2000)", "Large (2000-5000)"]

schools_size_df = school_summary_table
schools_size_df["School Size"] = pd.cut(student_counts, bins=bins, labels=size_ranges)

# Arrange scores by columns
spending_math_scores = schools_size_df.groupby(["School Size"])["Average Math Score"].mean()
spending_reading_scores = schools_size_df.groupby(["School Size"])["Average Reading Score"].mean()
spending_passing_math = schools_size_df.groupby(["School Size"])["% Passing Math"].mean()
spending_passing_reading = schools_size_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2

scores_by_size = school_summary_table[["School Size", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
scores_by_size = scores_by_size.groupby("School Size").mean()

scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (0-1000),83.820,83.92500,93.5500,96.10,94.82500
Medium (1000-2000),83.374,83.86800,93.6000,96.78,95.19000
Large (2000-5000),77.745,81.34375,69.9875,82.75,76.36875


## <font color=black> Scores By School Type </font>

In [120]:
# Using groupby, arrange scores by school type
scores_school_type = school_summary_table[["School Type","Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
scores_school_type = scores_school_type.groupby('School Type').mean()
scores_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.472500,83.897500,93.625000,96.575000,95.100000
District,76.955714,80.965714,66.571429,80.785714,73.678571
